In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            if a(j*dt) == 0:
                continue
            
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0] - prms[1]*t
    a2 = lambda t: (a(t))**2
    da2 = lambda t: -2*prms[1] * a(t)
    
    ndt = prms[2]
    mu = np.array([prms[3], prms[4], prms[5]])
    
    T_max = min(max(RT), prms[0]/prms[1])
    fpt = ie_bessel_fpt(a2, da2, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=T_max)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.0001 and rt - ndt < T_max:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = a(rt - ndt) * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'b0_true': [],
               'b0_estimate': [],
               'lambda_true': [],
               'lambda_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_b0 = 2
max_b0 = 5

min_lambda = .1
max_lambda = 2

min_ndt = 0.1
max_ndt = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(44)):
    b0 = np.random.uniform(min_b0, max_b0)
    lamb = np.random.uniform(min_lambda, max_lambda)
    a = lambda t: b0 - lamb*t
    
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                                             (min_ndt, max_ndt), (min_mu, max_mu),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_3D_likelihood,
                       args=(RT, Theta),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=[(min_b0, max_b0), (min_lambda, max_lambda),
                               (min_ndt, max_ndt), (min_mu, max_mu),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df['b0_true'].append(b0)
        recovery_df['lambda_true'].append(lamb)
        recovery_df['ndt_true'].append(ndt)
        recovery_df['mu1_true'].append(mu[0])
        recovery_df['mu2_true'].append(mu[1])
        recovery_df['mu3_true'].append(mu[2])

        recovery_df['b0_estimate'].append(min_ans.x[0])
        recovery_df['lambda_estimate'].append(min_ans.x[1])
        recovery_df['ndt_estimate'].append(min_ans.x[2])
        recovery_df['mu1_estimate'].append(min_ans.x[3])
        recovery_df['mu2_estimate'].append(min_ans.x[4])
        recovery_df['mu3_estimate'].append(min_ans.x[5])

100%|█████████████████████████████████████| 44/44 [10:56:48<00:00, 895.65s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
b0_true,1.000000,0.914158,-0.086475,-0.199853,0.113312,0.155594,-0.210013,-0.202529,-0.035735,-0.026703,-0.074700,-0.075138
b0_estimate,0.914158,1.000000,-0.223649,-0.129792,0.201810,0.159268,-0.090491,-0.083555,-0.037957,-0.034262,0.084475,0.078004
lambda_true,-0.086475,-0.223649,1.000000,0.857287,-0.205665,-0.135231,-0.102267,-0.103146,-0.057284,-0.051050,0.055515,0.063828
lambda_estimate,-0.199853,-0.129792,0.857287,1.000000,-0.111883,-0.136895,0.073011,0.073080,-0.000069,0.001349,0.251102,0.250586
ndt_true,0.113312,0.201810,-0.205665,-0.111883,1.000000,0.976404,-0.173258,-0.173487,-0.036366,-0.045824,0.061871,0.059399
ndt_estimate,0.155594,0.159268,-0.135231,-0.136895,0.976404,1.000000,-0.230822,-0.231033,-0.010312,-0.016773,-0.014233,-0.014130
mu1_true,-0.210013,-0.090491,-0.102267,0.073011,-0.173258,-0.230822,1.000000,0.998571,0.180185,0.175993,0.262560,0.237609
mu1_estimate,-0.202529,-0.083555,-0.103146,0.073080,-0.173487,-0.231033,0.998571,1.000000,0.171054,0.167947,0.246695,0.222174
mu2_true,-0.035735,-0.037957,-0.057284,-0.000069,-0.036366,-0.010312,0.180185,0.171054,1.000000,0.999108,0.200414,0.199876
mu2_estimate,-0.026703,-0.034262,-0.051050,0.001349,-0.045824,-0.016773,0.175993,0.167947,0.999108,1.000000,0.185444,0.185296


In [8]:
file_name = '_Recovery_data/IE_3d_recovery_linear_500_02.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,b0_true,b0_estimate,lambda_true,lambda_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.148317,2.359235,1.438196,1.663758,0.505154,0.485704,-2.370349,-2.445583,-1.277261,-1.316184,-2.054767,-2.091872
1,3.635062,3.439260,0.473456,0.436974,0.528541,0.552092,-0.374021,-0.417492,-2.106163,-2.025223,2.898536,2.858162
2,4.826623,4.532367,1.973275,1.786851,0.667942,0.711530,2.267376,2.340277,-0.491910,-0.526753,0.350809,0.376698
3,4.199072,3.893374,1.939724,2.000000,0.546225,0.621834,-0.195950,-0.277326,2.985417,3.000000,-0.283851,-0.328201
4,2.353982,2.409488,1.162493,1.237505,0.934116,0.938632,1.565139,1.498704,0.429792,0.449948,2.135110,2.060236
...,...,...,...,...,...,...,...,...,...,...,...,...
305,3.294678,3.802457,1.169142,1.588263,0.368535,0.327148,2.544877,2.532076,-0.053791,0.043349,-2.782369,-2.830676
306,2.289881,3.043749,0.996713,1.871520,0.865987,0.797816,2.934720,2.784183,2.131972,2.061403,2.998379,2.895582
307,2.823835,2.799800,1.503842,1.587440,0.766561,0.796163,0.934408,0.969617,-2.567493,-2.586019,-1.239156,-1.348382
308,3.095680,4.078288,0.307107,1.107127,0.587655,0.460154,2.392392,2.169708,0.425550,0.315742,2.979052,2.723277
